## A Neural Network for Time Series Classification

This chapter introduces a convolutional neural network architecture for time series classifiaction.

As a set of labelled time series, we are going use a [**heartbeat ECG dataset**](https://www.kaggle.com/shayanfazeli/heartbeat/). Each time series corresponds to an electrocardiogram (ECG) shapes of a heartbeat, and we have labelled examples for the normal case and abnormal cases (affected by different arrhythmias and myocardial infarction).

![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/9e/SinusRhythmLabels.svg/243px-SinusRhythmLabels.svg.png)  

_Schematic diagram of normal sinus rhythm for a human heart as seen on ECG_ [Source](https://commons.wikimedia.org/wiki/File:SinusRhythmLabels.svg#mw-jump-to-license)

## Preamble

In [ ]:
import matplotlib.pyplot as plt
import seaborn

In [ ]:
seaborn.set_style("ticks")
plt.rcParams["axes.grid"] = True

In [ ]:
import pandas
import numpy
import sklearn
import tensorflow

In [ ]:
import data_science_learning_paths

## Data

In [ ]:
data_science_learning_paths.datasets.download("heartbeat")

In [ ]:
ls ../.assets/data

In [ ]:
data, label =  data_science_learning_paths.datasets.read_heartbeat(balance_classes=True)

In [ ]:
data.head()

In [ ]:
label.head()

In [ ]:
ts_len = data.shape[1]
ts_len

### Exploration

In [ ]:
data[label == 0].sample(n=10).transpose().plot(title="examples of normal heartbeats")

In [ ]:
data[label == 1].sample(n=10).transpose().plot(title="examples of abnormal heartbeats")

## Classifier Architecture

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Convolution1D, Dense, Dropout, GlobalMaxPooling1D

In [ ]:
classifier = Sequential(
    [
        Input(
            shape=(1, 187)
        ),
        Convolution1D(
            filters=25,
            kernel_size=8,
            padding="same",
            activation="tanh"
        ),
        GlobalMaxPooling1D(),
        Dense(
            units=50,
            activation="tanh"
        ),
        Dropout(rate=0.3),
        Dense(
            units=1,
            activation="sigmoid"
        )
    ]
)

![](graphics/heartbeat-classifier.png)

In [ ]:
classifier.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

## Training

In [ ]:
data.head()

In order to evaluate the classifier later, we split off a random set of time series not used in the training.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_data, test_data, train_label, test_label = train_test_split(data, label, test_size=0.2, shuffle=True)

Bring the training data to the format expected by the classifier network:

In [ ]:
X_train = train_data.values.reshape(-1, 1, ts_len)
y_train = train_label.values

The network is now trained. Watch how the accuracy on the valiation set develops.

In [ ]:
classifier.fit(
    x=X_train,
    y=y_train,
    validation_split=0.2,
    batch_size=16,
    epochs=50,
)

### Evaluation

In [ ]:
X_test = test_data.values.reshape(-1, 1, ts_len)
y_test = test_label.values

In [ ]:
predicted = pandas.Series(
    classifier.predict_classes(X_test).flatten(),
    index=test_data.index
)


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(test_label, predicted)

### Exploration of Predictions

In [ ]:
text_label = lambda val: "abnormal" if val == 1 else "normal"

In [ ]:
for i in range(10):
    plt.figure()
    test_data.iloc[i].plot(kind="line")
    plt.suptitle(f"prediction: {text_label(predicted.iloc[i])} / truth: {text_label(test_label.iloc[i])}")

---
_This notebook is licensed under a [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/). Copyright © 2018-2021 [Point 8 GmbH](https://point-8.de)_